# Data Retrieval

In [1]:
import polars as pl
import polars.selectors as cs
from plotnine import *
import requests

In [13]:
res = []
url = 'https://www.federalregister.gov/api/v1/documents.json?fields[]=document_number&fields[]=excerpts&fields[]=page_length&fields[]=president&fields[]=publication_date&fields[]=raw_text_url&fields[]=signing_date&fields[]=title&fields[]=toc_subject&fields[]=topics&per_page=20&conditions[publication_date][gte]=2005-01-20&conditions[presidential_document_type][]=executive_order&conditions[president][]=george-w-bush&conditions[president][]=barack-obama&conditions[president][]=donald-trump&conditions[president][]=joe-biden&conditions[president][]=donald-trump'

while url != '':

    # retrieve results
    response = requests.get(url)
    response.raise_for_status()
    res_temp = response.json()
    res += [res_temp]

    # get next url
    url = res_temp.get('next_page_url','')

## Data Cleaning

In [14]:
df_res = (
    pl.DataFrame(res)
    .select('results')
    .explode('results')
    .unnest('results')
    .unnest('president')
)
df_res.glimpse()

In [24]:
# lookup table

term = [1,2,1,2,1,1,2]
start_year = [2001, 2005, 2009, 2013, 2017, 2021, 2025]
start_date = [f'{year}-01-20' for year in start_year]
end_date = start_date[1:] + ['2029-01-20']
president_lname = ['Bush']*2 + ['Obama']*2 + ['Trump'] + ['Biden'] + ['Trump']
df_lkup = pl.DataFrame([term, start_year, start_date, end_date, president_lname], 
                       schema = ['term','start_year','start_date', 'end_date', 'president_lname'],
                       orient = 'col')
df_lkup

In [26]:
df_clean = (
    df_res
    .with_columns(
        president_lname = pl.col('identifier')
                            .str.extract('[A-Za-z ]+-([A-Za-z]+$)', group_index = 1)
                            .str.to_titlecase(),
    )
    # there's a bug in polars join_asof() at the time of writing...
    .join(df_lkup, on = 'president_lname')
    .filter( pl.col('signing_date') >= pl.col('start_date'))
    .filter( pl.col('signing_date') <  pl.col('end_date'))
    .with_columns( pres_term = pl.concat_str( pl.col('president_lname'), pl.lit(' - '), pl.col('term')) )
)
df_clean.glimpse()

Rows: 952
Columns: 17
$ document_number         <str> '2025-14212', '2025-14217', '2025-14218', '2025-13925', '2025-12961', '2025-12962', '2025-12774', '2025-12775', '2025-12505', '2025-12506'
$ excerpts               <null> None, None, None, None, None, None, None, None, None, None
$ page_length             <i64> 4, 3, 3, 3, 2, 2, 3, 2, 1, 5
$ identifier              <str> 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump', 'donald-trump'
$ name                    <str> 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump'
$ publication_date        <str> '2025-07-28', '2025-07-28', '2025-07-28', '2025-07-23', '2025-07-10', '2025-07-10', '2025-07-09', '2025-07-09', '2025-07-03', '2025-07-03'
$ raw_text_url            <str> 'https://www.federalregister.gov/documents/full_text/text/2025/07/28/2

In [28]:
df_agg = (
    df_clean
    .group_by('pres_term')
    .agg(
        n_eo_issued = pl.col('document_number').count(),
        start_date = pl.col('publication_date').min()
    )
)

df_agg.sort('start_date')

In [59]:
df_cumul = (
    df_clean
    .with_columns( cs.ends_with('date').cast(pl.Date) )
    .with_columns(
        day_since_start = ( pl.col('signing_date') - pl.col('start_date') ).dt.total_days(),
        term_label = pl.concat_str( pl.lit('('), pl.col('start_date').dt.year(), pl.lit(')'),
                                    pl.lit(' '),
                                    pl.col('president_lname') )
    )
    .group_by('pres_term', 'term_label', 'day_since_start')
    .len()
    .with_columns( n_cumul = pl.col('len').cum_sum().over('pres_term', order_by = 'day_since_start') )
)
df_cumul.sort('pres_term', 'day_since_start').glimpse()

Rows: 711
Columns: 5
$ pres_term       <str> 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1'
$ term_label      <str> '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden'
$ day_since_start <i64> 0, 1, 2, 5, 6, 7, 8, 13, 15, 21
$ len             <u32> 9, 8, 2, 2, 1, 2, 1, 3, 1, 1
$ n_cumul         <u32> 9, 17, 19, 21, 22, 24, 25, 28, 29, 30

Rows: 711
Columns: 5
$ pres_term       <str> 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1', 'Biden - 1'
$ term_label      <str> '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden', '(2021) Biden'
$ day_since_start <i64> 0, 1, 2, 5, 6, 7, 8, 13, 15, 21
$ len             <u32> 9, 8, 2, 2, 1, 2, 1, 3, 1, 1
$ n_c

In [77]:
(
ggplot(df_cumul) +
aes(x = 'day_since_start', y = 'n_cumul', color = 'term_label') +
geom_line() +
labs(
    title = 'Executive Orders Signed by Days in Office',
    x = 'Days since Term Start', 
    y = 'EOs Issued', 
    color = 'Term') +
theme_538() +
scale_color_brewer('qual', palette = 'Set1') +
theme(legend_position = 'bottom')
)